In [92]:
#!pip install --upgrade pip
#!pip install transformers==3.1.0
#!pip install keybert
#!pip install -U imbalanced-learn
#!pip install --upgrade scikit-learn
#!pip install torch==1.2.0 torchvision==0.4.0

In [3]:
from transformers import pipeline
from os import listdir
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import f1_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
#from keybert import KeyBERT

In [ ]:
classifier = pipeline("zero-shot-classification",device=0) # to utilize GPU

We can use this pipeline by passing in a sequence and a list of candidate labels. The pipeline assumes by default that only one of the candidate labels is true, returning a list of scores for each label which add up to 1.

In [96]:
path_screenplays_scenes='coref/csi-corpus/screenplay_summarization/scene_level_n_aspects'
eps=[]


for ep in listdir(path_screenplays_scenes):
    annotated_scenes=pd.read_csv(path_screenplays_scenes+'/'+ep)
    eps.append(ep.split('.csv')[0])
    


centrality_dict = np.load('centralities.npy',allow_pickle=True)


#for ep in centrality_dict.item():
    #eps.append(ep)

#print(centrality_dict.item()['s05e17'][0].cpu())




In [97]:
def get_results_several_binary(df, classifier, topic_labels):
    for topic in topic_labels:
        df[topic] = None
        for i in df.index:
            #text = df.loc[i, 'scene_text']
            #text = df.loc[i, 'non_dialogue']
            text = df.loc[i, 'dialogue']
            if text is not None:
                #print(text)
                result = classifier(str(text),topic,multi_class=False)
                df.loc[i, topic] = result['scores'][0]

            
    return df

In [98]:
def add_centrality_score(df,centrality_scores):
    df['centrality']=None
    for i in range(len(centrality_scores)):
        #print(centrality_scores[i])
        df.loc[i, 'centrality'] = centrality_scores[i].cpu()
    return df

    

In [99]:

def f1_score_summer(in_summary,prediction_class):
    gs_summary = []
    re=[]
    pre=[]
    f1=[]
    for m, label in enumerate(first_results.in_summary):
        if label == 1:
            gs_summary.append(m)   
    extracted = []
    for m, label in enumerate(first_results.prediction_class):
        if label == 1:
            extracted.append(m)
    right_now = len(list(set(gs_summary).intersection(extracted)))
    overall_now = len(gs_summary)
    predicted_now = len(extracted)
    try:
        recall_now = right_now / overall_now
    except:
        recall_now = 0
    re.append(recall_now)
    try:
        precision_now = right_now / predicted_now
    except:
        precision_now = 0
    pre.append(precision_now)
    try:
        f1.append(
            (2 * precision_now * recall_now) / (precision_now + recall_now))
    except:
        f1.append(0)
    return f1

In [89]:

f1_scores =[]
f1_exactnb=[]
f1_30_perc=[]



for ep in eps:
    annotated_scenes=pd.read_csv(path_screenplays_scenes+'/'+ep+'.csv')
    
    # This paragraph separates dialogue and non dialogue
    dialogue_matches=annotated_scenes.scene_text.str.extractall(r"(?P<dialogue_matches>\[\[.*?\"\,)")
    annotated_scenes['dialogue']=annotated_scenes.scene_text.str.extract(r"(?P<dialogue>\[\[.*?\"\,)")
    annotated_scenes['non_dialogue']=annotated_scenes.scene_text.str.replace(r"(?P<non_dialogue>\[\[.*?\"\,)",'')
    annotated_scenes['dialogue']=annotated_scenes.dialogue.astype(str).str.replace('[^\w\s]','')
    annotated_scenes.non_dialogue = annotated_scenes.non_dialogue.str.replace('"', '')
    dialogue_matches.dialogue=dialogue_matches.dialogue_matches.groupby(level=0).apply(list)
    annotated_scenes.loc[annotated_scenes['dialogue'].notna(), 'dialogue'] = dialogue_matches.dialogue
    annotated_scenes.loc[annotated_scenes['dialogue'].isna(), 'dialogue'] = 'no dialogue'
    annotated_scenes.loc[annotated_scenes['non_dialogue'].isna(), 'non_dialogue'] = 'only dialogue'
    
    """#experience with keyword extraction
    non_dialogue_text= [' '.join(annotated_scenes['non_dialogue'])]
    #dialogue_text=[' '.join(annotated_scenes['dialogue'])]
    
    #kw_model = KeyBERT('distilbert-base-nli-mean-tokens')
    #keywords_non_dialogue = kw_model.extract_keywords(non_dialogue_text)
    #keywords_dialogue = kw_model.extract_keywords(dialogue_text)
    #print(keywords_non_dialogue)
    #print(keywords_dialogue)"""

    
    
    
    #candidate_labels=['Crime','Drama','Mystery','Thriller']
    #candidate_labels=["thought-provoking","storytelling","suspenseful"]
    #candidate_labels=['murder','death','blood','violence','pistol']
    #candidate_labels=['guilty','death','forensic']
    #candidate_labels=['crime scene', 'victim', 'death cause','evidence','motive','perpetrator']
    #candidate_labels=['motive','perpetrator']
    #candidate_labels=['death','killing','fear']
    #candidate_labels=['crime scene victim death cause perpetrator evidence motive killing police detective death murder']
    #candidate_labels=['crime']
    candidate_labels=['annihilate', 'annihilation', 'asphyxiate', 'assassin', 'assassinate', 'assassination', 'behead', 'beheading', 'blood-bath', 'bloodshed', 'butcher', 'butchery','carnage', 'crucifixion', 'crucify', 'deadly', 'decapitate', 'decapitation', 'destroy', 'dispatch','do in', 'drown', 'eliminate', 'euthanasia', 'euthanize', 'exterminate', 'extermination', 'fatal', 'fatality', 'fratricide', 'garrotte', 'genocide', 'holocaust', 'homicide','immolation', 'infanticide', 'kill', 'killer', 'killing', 'lethal', 'liquidate','liquidation','liquidator','lynch', 'massacre', 'matricide','murder', 'murderer', 'patricide', 'pogrom', 'regicide', 'shooting', 'silence', 'slaughter', 'slaughter', 'slaughterer', 'slay', 'slayer', 'slaying', 'smother', 'smothering', 'starve', 'suffocate', 'suffocation', 'suicide', 'take (someone)life', 'take out','terminate']
    #candidate_labels=['Killing','Buildings','Aggregate','Dead or Alive','Death','Locative relation','Attempt','Evidence','Natural features','Relative time']
    
    #candidate_labels=['deduction','character-driven drama', 'team of crime-scene investigators', 'Las Vegas Police Department', 'physical evidence', 'murders']
    #candidate_labels=['crime lab','police','crime scene investigation','las vegas nevada','detective']
    #candidate_labels=['police','detective','forensics','usa','investigation','evidence','nevada','investigator','suspicion','suspect','scene','crime','murder']
    
    #Computing total score (adding score of all classes) and sorting 
    first_results=get_results_several_binary(annotated_scenes,classifier,candidate_labels)
    first_results=add_centrality_score(first_results,centrality_dict.item()[ep])
    #print(first_results)
    first_results['prediction_class']=0
    cols_to_keep=['scene_id', 'scene_text', 'in_summary', 'aspects','prediction_class','non_dialogue','dialogue','centrality']
    first_results['score'] = first_results.drop(cols_to_keep, axis=1).max(axis=1)
    first_results[['score','centrality']]=MinMaxScaler().fit_transform(first_results[['score','centrality']])
    #first_results[['score','centrality']]=StandardScaler().fit_transform(first_results[['score','centrality']])
    #first_results['combined_score']=first_results['score']+first_results['centrality']
    first_results['combined_score']=first_results[['score','centrality']].max(axis=1)
    first_results=first_results.sort_values(by=['combined_score'],ascending=False)
    
    #first_results=first_results.sort_values(by=['score'],ascending=False)
    #first_results.to_csv('spoiler_detection_classes.csv')
    
   
    
    #Computing F1 score for different numbers of positive lables expected 
    first_results['prediction_class']=0
    first_results['prediction_class'].iloc[0:len(first_results[first_results.in_summary==1])]=1
    f1_exactnb.append(f1_score_summer(first_results.in_summary, first_results.prediction_class))
    first_results['prediction_class']=0
    first_results['prediction_class'].iloc[0:int(0.3*len(first_results.index))]=1
    f1_30_perc.append(f1_score_summer(first_results.in_summary, first_results.prediction_class))
    #print(first_results['prediction_class'].value_counts())
    #print(first_results[first_results.in_summary==1])


    
    
    
print(np.mean(f1_30_perc)) 


 


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

S

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Pandas doesn

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Pandas doesn

0.38748553270401664


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [268]:
pd.set_option('display.max_colwidth', -1)
print(first_results[first_results.in_summary==1])

    scene_id  \
32  32         
29  29         
27  27         
0   0          
12  12         
1   1          
24  24         
26  26         
19  19         
10  10         
8   8          
4   4          

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [ ]:
aspects_distribution={'motive': 0.109, 'evidence': 0.361,'cause of death': 0.116,'perpetrator': 0.144,'victim': 14.6,'crime scene': 0.124}
#candidate_labels={'death relative':100}
aspects_distribution={'cause of death': 0.116,'perpetrator': 0.144,'victim': 14.6,'crime scene': 0.124}






for topic,weight in tqdm(aspects_distribution.items()):
    f1_scores =[]
    for ep in eps:
        annotated_scenes=pd.read_csv(path_screenplays_scenes+'/'+ep+'.csv')

        # This paragraph separates dialogue and non dialogue
        dialogue_matches=annotated_scenes.scene_text.str.extractall(r"(?P<dialogue_matches>\[\[.*?\"\,)")
        annotated_scenes['dialogue']=annotated_scenes.scene_text.str.extract(r"(?P<dialogue>\[\[.*?\"\,)")
        annotated_scenes['non_dialogue']=annotated_scenes.scene_text.str.replace(r"(?P<non_dialogue>\[\[.*?\"\,)",'')
        annotated_scenes['dialogue']=annotated_scenes.dialogue.astype(str).str.replace('[^\w\s]','')
        annotated_scenes.non_dialogue = annotated_scenes.non_dialogue.str.replace('"', '')
        dialogue_matches.dialogue=dialogue_matches.dialogue_matches.groupby(level=0).apply(list)
        annotated_scenes.loc[annotated_scenes['dialogue'].notna(), 'dialogue'] = dialogue_matches.dialogue
        annotated_scenes.loc[annotated_scenes['dialogue'].isna(), 'dialogue'] = 'no dialogue'
        annotated_scenes.loc[annotated_scenes['non_dialogue'].isna(), 'non_dialogue'] = 'only dialogue'

        """#experience with keyword extraction
        non_dialogue_text= [' '.join(annotated_scenes['non_dialogue'])]
        #dialogue_text=[' '.join(annotated_scenes['dialogue'])]

        #kw_model = KeyBERT('distilbert-base-nli-mean-tokens')
        #keywords_non_dialogue = kw_model.extract_keywords(non_dialogue_text)
        #keywords_dialogue = kw_model.extract_keywords(dialogue_text)
        #print(keywords_non_dialogue)
        #print(keywords_dialogue)"""




        #candidate_labels=['Crime','Drama','Mystery','Thriller']
        #candidate_labels=["thought-provoking","storytelling","suspenseful"]
        #candidate_labels=['murder','death','blood','violence','pistol']
        #candidate_labels=['guilty','death','forensic']
        #candidate_labels=['crime scene', 'victim', 'death cause','evidence','motive','perpetrator']
        #candidate_labels=['motive','perpetrator']
        #candidate_labels=['death','killing','fear']
        #candidate_labels=['crime scene victim death cause perpetrator evidence motive killing police detective death murder']
        candidate_labels=topic
        #candidate_labels=['Killing','Buildings','Aggregate','Dead or Alive','Death','Locative relation','Attempt','Evidence','Natural features','Relative time']

        #candidate_labels=['crime scene', 'victim', 'death cause', 'perpetrator', 'evidence', 'motive']
        #candidate_labels=['deduction','character-driven drama', 'team of crime-scene investigators', 'Las Vegas Police Department', 'physical evidence', 'murders']
        #candidate_labels=['crime lab','police','crime scene investigation','las vegas nevada','detective']
        #candidate_labels=['police','detective','forensics','usa','investigation','evidence','nevada','investigator','suspicion','suspect','scene','crime','murder']

        #Computing total score (adding score of all classes) and sorting 
        first_results=get_results_several_binary(annotated_scenes,classifier,candidate_labels)
        first_results=add_centrality_score(first_results,centrality_dict.item()[ep])
        #print(first_results)
        first_results['prediction_class']=0
        cols_to_keep=['scene_id', 'scene_text', 'in_summary', 'aspects','prediction_class','non_dialogue','dialogue','centrality']
        first_results['score'] = first_results.drop(cols_to_keep, axis=1).max(axis=1)
        first_results[['score','centrality']]=MinMaxScaler().fit_transform(first_results[['score','centrality']])
        #first_results[['score','centrality']]=StandardScaler().fit_transform(first_results[['score','centrality']])
        #first_results['combined_score']=first_results['score']+first_results['centrality']
        first_results['combined_score']=first_results[['score','centrality']].max(axis=1)
        first_results=first_results.sort_values(by=['combined_score'],ascending=False)

        #first_results=first_results.sort_values(by=['score'],ascending=False)
        #first_results.to_csv('spoiler_detection_classes.csv')



        #Computing F1 score for different numbers of positive lables expected 
        first_results['prediction_class']=0
        first_results['prediction_class'].iloc[0:len(first_results[first_results.in_summary==1])]=1
        f1_exactnb.append(f1_score_summer(first_results.in_summary, first_results.prediction_class))
        first_results['prediction_class']=0
        first_results['prediction_class'].iloc[0:int(weight*len(first_results.index))]=1
        f1_scores.append(f1_score_summer(first_results.in_summary, first_results.prediction_class))
        #print(first_results['prediction_class'].value_counts())
        #print(first_results[first_results.in_summary==1])




    first_results.to_csv('{}_aspect.csv'.format(topic))
    print(np.mean(f1_scores)) 

    

  0%|          | 0/4 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access


In [ ]:
#eps=['s05e10']
for ep in eps:
    print(ep)
    df=pd.read_csv(ep+'_'+method_used+'_similarity.csv')
    print(df)
    

In [6]:
#print(first_results[first_results.in_summary==1])


print(first_results[['aspects','in_summary','topic','score','prediction_class','sum_score','prediction','prediction_score']])



NameError: name 'first_results' is not defined

In [ ]:

f1_scores =[]
f1_exactnb=[]
f1_30_perc=[]



for ep in eps:
    annotated_scenes=pd.read_csv(path_screenplays_scenes+'/'+ep+'.csv')
    
    # This paragraph separates dialogue and non dialogue
    dialogue_matches=annotated_scenes.scene_text.str.extractall(r"(?P<dialogue_matches>\[\[.*?\"\,)")
    annotated_scenes['dialogue']=annotated_scenes.scene_text.str.extract(r"(?P<dialogue>\[\[.*?\"\,)")
    annotated_scenes['non_dialogue']=annotated_scenes.scene_text.str.replace(r"(?P<non_dialogue>\[\[.*?\"\,)",'')
    annotated_scenes['dialogue']=annotated_scenes.dialogue.astype(str).str.replace('[^\w\s]','')
    annotated_scenes.non_dialogue = annotated_scenes.non_dialogue.str.replace('"', '')
    dialogue_matches.dialogue=dialogue_matches.dialogue_matches.groupby(level=0).apply(list)
    annotated_scenes.loc[annotated_scenes['dialogue'].notna(), 'dialogue'] = dialogue_matches.dialogue
    annotated_scenes.loc[annotated_scenes['dialogue'].isna(), 'dialogue'] = 'no dialogue'
    annotated_scenes.loc[annotated_scenes['non_dialogue'].isna(), 'non_dialogue'] = 'only dialogue'
    
    """#experience with keyword extraction
    non_dialogue_text= [' '.join(annotated_scenes['non_dialogue'])]
    #dialogue_text=[' '.join(annotated_scenes['dialogue'])]
    
    #kw_model = KeyBERT('distilbert-base-nli-mean-tokens')
    #keywords_non_dialogue = kw_model.extract_keywords(non_dialogue_text)
    #keywords_dialogue = kw_model.extract_keywords(dialogue_text)
    #print(keywords_non_dialogue)
    #print(keywords_dialogue)"""

    
    
    
    #candidate_labels=['Crime','Drama','Mystery','Thriller']
    #candidate_labels=["thought-provoking","storytelling","suspenseful"]
    #candidate_labels=['murder','death','blood','violence','pistol']
    #candidate_labels=['guilty','death','forensic']
    #candidate_labels=['crime scene', 'victim', 'death cause','evidence','motive','perpetrator']
    #candidate_labels=['motive','perpetrator']
    #candidate_labels=['death','killing','fear']
    #candidate_labels=['crime scene victim death cause perpetrator evidence motive killing police detective death murder']
    candidate_labels=['thriller']
    #candidate_labels=['Killing','Buildings','Aggregate','Dead or Alive','Death','Locative relation','Attempt','Evidence','Natural features','Relative time']
    
    #candidate_labels=['crime scene', 'victim', 'death cause', 'perpetrator', 'evidence', 'motive']
    #candidate_labels=['deduction','character-driven drama', 'team of crime-scene investigators', 'Las Vegas Police Department', 'physical evidence', 'murders']
    #candidate_labels=['crime lab','police','crime scene investigation','las vegas nevada','detective']
    #candidate_labels=['police','detective','forensics','usa','investigation','evidence','nevada','investigator','suspicion','suspect','scene','crime','murder']
    
    #Computing total score (adding score of all classes) and sorting 
    first_results=get_results_several_binary(annotated_scenes,classifier,candidate_labels)
    first_results=add_centrality_score(first_results,centrality_dict.item()[ep])
    #print(first_results)
    first_results['prediction_class']=0
    cols_to_keep=['scene_id', 'scene_text', 'in_summary', 'aspects','prediction_class','non_dialogue','dialogue','centrality']
    first_results['score'] = first_results.drop(cols_to_keep, axis=1).max(axis=1)
    first_results[['score','centrality']]=MinMaxScaler().fit_transform(first_results[['score','centrality']])
    #first_results[['score','centrality']]=StandardScaler().fit_transform(first_results[['score','centrality']])
    #first_results['combined_score']=first_results['score']+first_results['centrality']
    first_results['combined_score']=first_results[['score','centrality']].max(axis=1)
    first_results=first_results.sort_values(by=['score'],ascending=False)
    
    #first_results=first_results.sort_values(by=['score'],ascending=False)
    #first_results.to_csv('spoiler_detection_classes.csv')
    
   
    
    #Computing F1 score for different numbers of positive lables expected 
    first_results['prediction_class']=0
    first_results['prediction_class'].iloc[0:len(first_results[first_results.in_summary==1])]=1
    f1_exactnb.append(f1_score_summer(first_results.in_summary, first_results.prediction_class))
    first_results['prediction_class']=0
    first_results['prediction_class'].iloc[0:int(0.3*len(first_results.index))]=1
    f1_30_perc.append(f1_score_summer(first_results.in_summary, first_results.prediction_class))
    #print(first_results['prediction_class'].value_counts())
    #print(first_results[first_results.in_summary==1])


    
    
    
print(np.mean(f1_30_perc)) 


 


In [40]:
print(np.mean(f1_scores))

print(len(f1_scores))



0.2940651856941449
39


In [ ]:
f1_scores_bis=[]





for ep in eps:
    df=pd.read_csv('combined_scores_murders'+ep+'.csv')
    #df['max']=df[["prediction_score", "score_hf"]].max(axis=1)
    df['max']=0.5*df['score_hf']+0.5*df['prediction_score']
    #df['max'][df.topic=='joy']=0
    df=df.sort_values(by=['prediction_score'],ascending=False)
    df['prediction_class']=0
    df['prediction_class'].iloc[0:len(df[df.in_summary==1])]=1
    #print(df[['in_summary','topic','score_hf','prediction_class','sum_score','prediction','prediction_score','max']])
    
            
            
    #first_results.to_csv('combined_scores'+ep+'.csv')
    #first_results.loc[first_results['scene_id'].isin(predicted_pos), 'prediction_huggingface'] = 1 
    #print(first_results)
    df=df.sort_values(by=['score_hf'],ascending=False)
    df['prediction_class'].iloc[0:len(df[df.in_summary==1])]=1
    #first_results.loc[first_results['scene_id'].isin(predicted_pos), 'prediction_huggingface'] = 1 
    #print(first_results0
    f1_scores_bis.append(f1_score(df.in_summary, df.prediction_class))
    


print(np.mean(f1_scores_bis))

#print(len(f1_scores))




In [6]:

f1_scores_emotions=[]
for ep in eps:
    df=pd.read_csv(ep+'_'+method_used+'_similarity.csv')
    sequence=df.scene_text
    #candidate_labels = ['Crime scene', 'Victim', 'Death cause', 'Perpetrator', 'Evidence', 'Motive']
    candidate_labels=['anger', 'anticipation', 'joy', 'trust', 'fear', 'surprise', 'sadness', 'disgust']
    first_results=get_results(df,classifier,candidate_labels)   
    first_results['prediction_class']=0
    first_results[first_results.topic=='Crime'].sort_values(by=['score'],ascending=False)[]

    ranked['prediction_class'].iloc[0:len(first_results[first_results.in_summary==1])]=1
    ranked.to_csv('murder_hum_class'+ep+'.csv')
    print(ranked)
    f1_scores_emotions.append(f1_score(ranked.in_summary, ranked.prediction_class))
   
classifier(sequence, candidate_labels, multi_class=True)


SyntaxError: invalid syntax (<ipython-input-6-c61821b32eb2>, line 9)

In [114]:
import numpy as np

sorted_f1 = f1_scores.copy()
sorted_f1.sort()
print(f1_scores[10])
print(eps[10])
ep=eps[10]
print(sorted_f1)
df=pd.read_csv('murder_hum_class'+ep+'.csv')

#print(df.scene_text[df.in_summary==0].iloc[4])
print(df)


1.0
s04e09
[0.19999999999999998, 0.2857142857142857, 0.28571428571428575, 0.3157894736842105, 0.3333333333333333, 0.36363636363636365, 0.4, 0.4, 0.4285714285714285, 0.43478260869565216, 0.45454545454545453, 0.47058823529411764, 0.5, 0.5, 0.5, 0.5, 0.5000000000000001, 0.5714285714285714, 0.6, 0.6, 0.6, 0.6153846153846153, 0.6153846153846153, 0.6153846153846153, 0.631578947368421, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.7000000000000001, 0.7142857142857143, 0.7142857142857143, 0.7272727272727273, 0.7272727272727273, 0.7272727272727273, 0.7499999999999999, 0.7499999999999999, 0.8571428571428571, 1.0]
   Unnamed: 0  Unnamed: 0.1  scene_id  \
0           8             8         8   
1          30            30        30   

                                          scene_text  in_summary      aspects  \
0  [' ( Scene opens on a close up of two prints ,...           1  Death cause   
1  [' ( CATHERINE puts the bagged vice grips on t...           1   

In [4]:
df_all=pd.DataFrame(columns=['Unnamed: 0', 'scene_id', 'scene_text', 'in_summary', 'aspects',
       'prediction_score', 'prediction', 'aspect?'])
f1_scores=[]
eps=['s05e10']
for ep in eps:
    print(ep)
    df=pd.read_csv(ep+'_'+method_used+'_similarity.csv')
    print(df.columns)
    df_all=df_all.append(df)
    
   


s05e10
Index(['Unnamed: 0', 'scene_id', 'scene_text', 'in_summary', 'aspects',
       'prediction_score', 'prediction', 'aspect?'],
      dtype='object')


In [ ]:

eps=[]

for ep in listdir(path_screenplays_scenes):
    annotated_scenes=pd.read_csv(path_screenplays_scenes+'/'+ep)
    eps.append(ep.split('.csv')[0])

print(eps)
    
tags_entire_episode=[]


eps=['s03e19', 's04e14', 's04e06', 's01e07', 's04e21', 's05e17', 's03e08', 's04e10', 's04e05', 's05e05', 's05e06', 's01e20', 's04e15', 's02e06', 's01e08', 's04e12', 's05e08', 's03e05', 's02e10', 's05e13', 's03e11', 's01e13']

for ep in eps:
    print(ep)
    df=pd.read_csv(ep+'_'+method_used+'_similarity.csv')
    #print(df)
    #candidate_labels = ['Crime scene', 'Victim', 'Death cause', 'Perpetrator', 'Evidence', 'Motive']
    #candidate_labels=['crime scene', 'victim', 'death cause', 'perpetrator', 'evidence', 'motive']
    #candidate_labels=['kill','love']
    #action, drama, horror and romance.
    #candidate_labels=['action', 'crime','romance','comedy','murder']
    #candidate_labels=['action', 'crime','romance','comedy']
    #candidate_labels=joy,sadness, anger,fear, trust,distrust, surprise,anticipation
    candidate_labels=['murder','violence', 'flashback', 'romantic', 'cult', 'revenge', 'psychedelic', 'comedy', 'suspenseful', 'good_versus_evil', 'humor', 'satire', 'entertaining', 'neo noir', 'action', 'sadist', 'insanity', 'tragedy', 'fantasy', 'paranormal', 'boring', 'mystery', 'horror', 'melodrama', 'cruelty', 'gothic', 'dramatic', 'dark', 'atmospheric', 'sci-fi', 'psychological', 'historical', 'absurd', 'prank', 'sentimental', 'philosophical', 'avant garde', 'bleak', 'alternate reality', 'depressing', 'plot twist', 'realism', 'cute', 'stupid', 'intrigue', 'pornographic', 'home movie', 'haunting', 'historical fiction', 'allegory', 'adult comedy', 'inspiring', 'anti war', 'comic', 'brainwashing', 'alternate history', 'queer', 'clever', 'claustrophobic', 'whimsical', 'feel-good', 'blaxploitation', 'western', 'grindhouse film', 'magical realism', 'suicidal', 'autobiographical', 'christian film', 'non fiction']
    df['topic'] = None
    for i in tqdm(range(0, df.shape[0])):
        df.loc[i, 'topic']='bigoudi'
        text = df.loc[i, 'scene_text']
        result= classifier(text, 'murder')
        if text is not None:
            for j in candidate_labels:
                result_provisoire = classifier(text, j)
                #print(result_provisoire['scores'][0].type)
                #print(result_provisoire['scores'][0])
                #print(result['scores'][0])
                if result_provisoire['scores'][0]>result['scores'][0]:
                    result['scores']=result_provisoire['scores']
                    #print(result['scores'])
                    df.loc[i, 'topic'] = result_provisoire['labels'][0]
                    #print(df.loc[i, 'topic'])                   
            #print(result['labels'][0])
    #print(df.loc[i, 'topic'])
    print(df['topic'].value_counts().argmax())
    tags_entire_episode.append(df['topic'].value_counts().argmax())
    
    
   

In [ ]:


    sequence=df.scene_text
    #candidate_labels = ['Crime scene', 'Victim', 'Death cause', 'Perpetrator', 'Evidence', 'Motive']
    #candidate_labels=['crime scene', 'victim', 'death cause', 'perpetrator', 'evidence', 'motive']
    #candidate_labels=['kill']
    #action, drama, horror and romance.
    #candidate_labels=['action', 'crime','romance','comedy']
    #candidate_labels=['action', 'crime','romance','comedy']
    #candidate_labels=joy,sadness, anger,fear, trust,distrust, surprise,anticipation
    candidate_labels=['surprise','anticipation']
    
    #candidate_labels=['Action','Animation','Biography','Comedy','Crime','Drama','Family','Fantasy','Horror','Mystery','Romance','SciFi','Thriller']
    #candidate_labels=['Action','Biography','Comedy','Crime','Drama','Family','Fantasy','Horror','Romance','SciFi','Thriller']
    #candidate_labels=['anger', 'sadness','joy',feat]
    first_results=get_results(df,classifier,candidate_labels)
    first_results[['prediction_score','score_hf']]=MinMaxScaler().fit_transform(first_results[['prediction_score','score_hf']])
                  
    #print(first_results)
    first_results['sum_score']=0.5*first_results['score_hf']+0.5*first_results['prediction_score']
    #print(first_results['sum_score'])
    #ranked=first_results[first_results.topic=='murder'].sort_values(by=['score'],ascending=False)
    first_results['prediction_class']=0
    #ranked['prediction_class']=0
    #ranked['prediction_class'].iloc[0:len(first_results[first_results.in_summary==1])]=1
    #predicted_pos= ranked['scene_id'][ranked['prediction_class']==1]
    #predicted_pos=list(predicted_pos)
    #print(predicted_pos)
    #first_results=first_results.sort_values(by=['sum_score'],ascending=False)
    first_results=first_results.sort_values(by=['score_hf'],ascending=False)
    first_results['prediction_class'].iloc[0:len(first_results[first_results.in_summary==1])]=1
    #first_results.to_csv('combined_scores_kill'+ep+'.csv',index=None)
    #first_results.loc[first_results['scene_id'].isin(predicted_pos), 'prediction_huggingface'] = 1 
    #print(first_results)
    f1_scores.append(f1_score(first_results.in_summary, first_results.prediction_class))
    

In [ ]:
def get_results(df, classifier, topic_labels):
    df['topic'] = None
    df['score'] = None
    
    for i in tqdm(range(0, df.shape[0])):
        text = df.loc[i, 'scene_text']
        if text is not None:
            result = classifier(text, topic_labels)
            #df.loc[i, 'topic'] = str(str(result['labels'][0])+"+"+str(result['labels'][1]))
            #df.loc[i, 'score'] = str(str(result['scores'][0])+"+"+str(result['scores'][1]) 
            df.loc[i, 'topic'] = result['labels'][0]
            df.loc[i, 'score'] = result['scores'][0]
            #if result['labels'][0]=='surprise':
                #df.loc[i, 'score_hf'] = result['scores'][0]
            #else:
                #df.loc[i, 'score_hf'] = -1
            
    return df


To do multi-class classification, simply pass `multi_class=True`. In this case, the scores will be independent, but each will fall between 0 and 1.

In [ ]:
sequence = "Who are you voting for in 2020?"
candidate_labels = ["politics", "public health", "economics", "elections"]

classifier(sequence, candidate_labels, multi_class=True)

{'labels': ['politics', 'elections', 'public health', 'economics'],
 'scores': [0.972069501876831,
  0.967610776424408,
  0.03248710557818413,
  0.0061644683592021465],
 'sequence': 'Who are you voting for in 2020?'}

Here's an example of sentiment classification: 

In [ ]:
sequence = "I hated this movie. The acting sucked."
candidate_labels = ["positive", "negative"]

classifier(sequence, candidate_labels)

{'labels': ['negative', 'positive'],
 'scores': [0.9916268587112427, 0.00837317667901516],
 'sequence': 'I hated this movie. The acting sucked.'}

So how does this method work?

The underlying model is trained on the task of Natural Language Inference (NLI), which takes in two sequences and determines whether they contradict each other, entail each other, or neither.

This can be adapted to the task of zero-shot classification by treating the sequence which we want to classify as one NLI sequence (called the premise) and turning a candidate label into the other (the hypothesis). If the model predicts that the constructed premise _entails_ the hypothesis, then we can take that as a prediction that the label applies to the text. Check out [this blog post](https://joeddav.github.io/blog/2020/05/29/ZSL.html) for a more detailed explanation.

By default, the pipeline turns labels into hypotheses with the template `This example is {class_name}.`. This works well in many settings, but you can also customize this for your specific setting. Let's add another review to our above sentiment classification example that's a bit more challenging:

In [ ]:
sequences = [
    "I hated this movie. The acting sucked.",
    "This movie didn't quite live up to my high expectations, but overall I still really enjoyed it."
]
candidate_labels = ["positive", "negative"]

classifier(sequences, candidate_labels)

[{'labels': ['negative', 'positive'],
  'scores': [0.9916267991065979, 0.008373182266950607],
  'sequence': 'I hated this movie. The acting sucked.'},
 {'labels': ['negative', 'positive'],
  'scores': [0.8148515820503235, 0.1851484179496765],
  'sequence': "This movie didn't quite live up to my high expectations, but overall I still really enjoyed it."}]

The second example is a bit harder. Let's see if we can improve the results by using a hypothesis template which is more specific to the setting of review sentiment analysis. Instead of the default, `This example is {}.`, we'll use, `The sentiment of this review is {}.` (where `{}` is replaced with the candidate class name)

In [ ]:
sequences = [
    "I hated this movie. The acting sucked.",
    "This movie didn't quite live up to my high expectations, but overall I still really enjoyed it."
]
candidate_labels = ["positive", "negative"]
hypothesis_template = "The sentiment of this review is {}."

classifier(sequences, candidate_labels, hypothesis_template=hypothesis_template)

[{'labels': ['negative', 'positive'],
  'scores': [0.9890093207359314, 0.010990672744810581],
  'sequence': 'I hated this movie. The acting sucked.'},
 {'labels': ['positive', 'negative'],
  'scores': [0.9581228494644165, 0.0418771356344223],
  'sequence': "This movie didn't quite live up to my high expectations, but overall I still really enjoyed it."}]

By providing a more precise hypothesis template, we are able to see a more accurate classification of the second review.

> Note that sentiment classification is used here just as an illustrative example. The [Hugging Face Model Hub](https://huggingface.co/models?filter=text-classification) has a number of models trained specifically on sentiment tasks which can be used instead.

#### Update: Zero-shot classification in 100 languages

Interested in using the pipeline for languages other than English? We've trained a cross-lingual model on top of XLM RoBERTa which you can use by passing `model='joeddav/xlm-roberta-large-xnli'` when creating the pipeline:

In [ ]:
classifier = pipeline("zero-shot-classification", model='joeddav/xlm-roberta-large-xnli')

You can use it with any combination of languages. For example, let's classify a Russian sentence with English candidate labels:

In [ ]:
sequence = "За кого вы голосуете в 2020 году?" # translation: "Who are you voting for in 2020?"
candidate_labels = ["Europe", "public health", "politics"]

classifier(sequence, candidate_labels)

{'labels': ['politics', 'Europe', 'public health'],
 'scores': [0.9048484563827515, 0.05722189322113991, 0.03792969882488251],
 'sequence': 'За кого вы голосуете в 2020 году?'}

Now let's do the same but with the labels in French:



In [ ]:
sequence = "За кого вы голосуете в 2020 году?" # translation: "Who are you voting for in 2020?"
candidate_labels = ["Europe", "santé publique", "politique"]

classifier(sequence, candidate_labels)

{'labels': ['politique', 'Europe', 'santé publique'],
 'scores': [0.9726154804229736, 0.017128489911556244, 0.010256024077534676],
 'sequence': 'За кого вы голосуете в 2020 году?'}

As we discussed in the last section, the default hypothesis template is the English, `This text is {}.`. If you are working strictly within one language, it may be worthwhile to translate this to the language you are working with:

In [ ]:
sequence = "¿A quién vas a votar en 2020?"
candidate_labels = ["Europa", "salud pública", "política"]
hypothesis_template = "Este ejemplo es {}."

classifier(sequence, candidate_labels, hypothesis_template=hypothesis_template)

{'labels': ['política', 'Europa', 'salud pública'],
 'scores': [0.9109585881233215, 0.05954807624220848, 0.029493311420083046],
 'sequence': '¿A quién vas a votar en 2020?'}

The model is fine-tuned on XNLI which includes 15 languages: Arabic, Bulgarian, Chinese, English, French, German, Greek, Hindi, Russian, Spanish, Swahili, Thai, Turkish, Urdu, and Vietnamese. The base model is trained on 85 more, so the model will work to some degree for any of those in the XLM RoBERTa training corpus (see the full list in appendix A of the [XLM Roberata paper](https://arxiv.org/abs/1911.02116)).

See the [model page](https://huggingface.co/joeddav/xlm-roberta-large-xnli) for more.